In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam

df = pd.read_excel("default of credit card clients.xls", skiprows=1)
df.rename(columns={"default payment next month": "target"}, inplace=True)
df.drop(columns=['ID'], inplace=True)

X = df.drop(columns=['target']).values.astype(np.float32)
y = df['target'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_seq = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

inputs = Input(shape=(1, X_train.shape[1]))
x = Bidirectional(LSTM(60, return_sequences=True))(inputs)
x = Dropout(0.4)(x)
x = Bidirectional(LSTM(80, return_sequences=True))(x)
x = Dropout(0.4)(x)
x = Bidirectional(LSTM(120, return_sequences=True))(x)
x = Dropout(0.4)(x)
x = GlobalMaxPooling1D()(x)
outputs = Dense(1, activation='sigmoid')(x)

bdlstm_model = Model(inputs, outputs)
bdlstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

bdlstm_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

feature_extractor = Model(inputs=bdlstm_model.input, outputs=x)
X_train_feat = feature_extractor.predict(X_train_seq)
X_test_feat = feature_extractor.predict(X_test_seq)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_feat, y_train)

y_pred = xgb.predict(X_test_feat)
y_prob = xgb.predict_proba(X_test_feat)[:, 1]

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📈 F1 Score:", f1_score(y_test, y_pred))
print("🏆 ROC AUC:", roc_auc_score(y_test, y_prob))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.7957 - loss: 0.4869 - val_accuracy: 0.8087 - val_loss: 0.4630
Epoch 2/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8099 - loss: 0.4632 - val_accuracy: 0.8123 - val_loss: 0.4590
Epoch 3/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8136 - loss: 0.4541 - val_accuracy: 0.8085 - val_loss: 0.4626
Epoch 4/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8143 - loss: 0.4512 - val_accuracy: 0.8133 - val_loss: 0.4558
Epoch 5/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8165 - loss: 0.4472 - val_accuracy: 0.8098 - val_loss: 0.4534
Epoch 6/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8125 - loss: 0.4522 - val_accuracy: 0.8131 - val_loss: 0.4650
Epoch 7/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8105 - loss: 0.4557 - val_accuracy: 0.8133 - val_loss: 0.4582
Epoch 8/50
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8162 - loss: 0.4440 - val_accuracy: 0

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:52:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Accuracy: 0.8103333333333333
📈 F1 Score: 0.4560229445506692
🏆 ROC AUC: 0.7464290442731587

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.94      0.89      4673
           1       0.62      0.36      0.46      1327

    accuracy                           0.81      6000
   macro avg       0.73      0.65      0.67      6000
weighted avg       0.79      0.81      0.79      6000

